In [2]:
# e = 65536
# e*d % phi(n) = 1
# e*e^(phi(phi(n))-1) = 1 % phi(n)
# d = e^(phi(phi(n))-1)
# ref: https://doc.sagemath.org/html/en/thematic_tutorials/numtheory_rsa.html
class RSA:
    def __init__(self, N):
        #p1 = random_prime(N)
        #p2 = random_prime(N)
        p1 = random_prime(N,1,N/2)
        p2 = random_prime(N,1,N/2)
        n = p1*p2
        phi = (p1-1)*(p2-1)
        e = Integers().random_element(phi)
        while gcd(e, phi)!=1:
            e = Integers().random_element(phi)
        self.e = e
        bezout = xgcd(e,phi)
        d = Integer(mod(bezout[1], phi))
        self.d = d
    def gen_key(self):
        return (self.e, self.d)

In [30]:
# for simple case, let H(m) = m
def H(m):
    return m

In [31]:
# To realize DLP-based ch, I have to add a RSA encrypt (p,q)
# ref: Identity-Based Chameleon Hash and Applications
# G. Ateniese
class chameleon_hash:
    def __init__(self, Zq, q, N):
        self.Zq = Zq
        self.q = q
        self.N = N
    def Gen(self):
        rsa = RSA(self.N)
        e, t = rsa.gen_key()
        I =  self.Zq(randint(1, self.q))
        pk = [I, e]
        rho = self.Zq(randint(1, self.q))
        return (pk, rho, t)
    
    def CH(self, pk, m, rho):
        return pk[0]^H(m)*rho^pk[1]
    
    def UF(self, pk, t, m, rho, m_):
        B = pk[0]^t
        rho_ = rho*B^(H(m) - H(m_))
        return rho_

In [32]:
class signature_oracle:
    def __init__(self, g, q, N):
        self.g = g
        self.sk = randint(1, N)
    def Sign(self, m):
        return (m, self.g^self.sk)
    def Verify(self, m, sigma):
        return (sigma==self.Sign(m))

In [33]:
# has secret x
class Prover:
    def __init__(self, g, q, N):
        self.honest = True
        self.g = g
        self.k = randint(1, q)
        self.x = -99999
        
    def Commit(self, S, p_set):
        r_ = self.g^self.k
        pk = p_set[0]
        CHF = p_set[1]
        rho = p_set[2]
        r = CHF.CH(pk, r_, rho)
        return (r_, r, S.Sign(r))
    
    def Prove(self, S, c):
        s = self.k+c*self.x
        return (s, S.Sign(s))

In [34]:
class Verifier:
    def __init__(self, Zq, g, q, N):
        self.g = g
        self.q = q
        self.t = -99999
        self.CHF = chameleon_hash(Zq, q, N)
        
    def KeyGen(self):
        return self.CHF.Gen()
    
    def Challenge(self):
        c = randint(1,self.q)
        return c
    
    def Verify(self, S, pk, rho, y, r, c, s, sigma_r, sigma_s):
        v1 = S.Verify(r, sigma_r)
        v2 = S.Verify(s, sigma_s)
        rh_ = self.g^s * y^(-c)
        rh = self.CHF.CH(pk, rh_, rho)
        v3 = (r==rh)
        return (v1 and v2 and v3)

In [35]:
import time
class NTZKP_Protocol:
    def __init__(self, bits, seed):
        set_random_seed(seed)
        current_randstate().set_seed_gp()
        
        N = 2^bits
        q = random_prime(N)
        Zq = Integers(q)
        g = Zq(2)
        
        # knowledge
        x = randint(1, q)
        # such that
        y = g^x
        
        prover = Prover(g, q, N)
        verifier = Verifier(Zq, g, q, N)
        S = signature_oracle(g, q, N)
        
        if prover.honest:
            prover.x = x
        else:
            prover.x = randint(1, q)
        
        start = time.time()
        pk, rho, verifier.t = verifier.KeyGen()
        end = time.time()
        self.keygen_time = end - start
        
        p_set = [pk, verifier.CHF, rho]
        
        start = time.time()
        r_, r, sigma_r = prover.Commit(S, p_set)
        end = time.time()
        self.commit_time = end - start
        
        start = time.time()
        c = verifier.Challenge()
        end = time.time()
        self.challenge_time = end - start
        
        start = time.time()
        s, sigma_s = prover.Prove(S, c)
        end = time.time()
        self.prove_time = end-start
        
        start = time.time()
        self.result = verifier.Verify(S, pk, rho, y, r, c, s, sigma_r, sigma_s)
        end = time.time()
        self.verify_time = end - start

In [41]:
bits = 2048
ctr = 0
iteration = 20
total_keygen_time = 0
total_commit_time = 0
total_challenge_time = 0
total_prove_time = 0
total_verify_time = 0

while ctr < iteration:
    seed = time.time_ns()
    print("iteration : %d / %d" % (ctr+1, iteration), end="\r")
    
    ntzkp = NTZKP_Protocol(bits, seed)

    if ntzkp.result:
        ctr += 1
        total_keygen_time += ntzkp.keygen_time*10^6
        total_commit_time += ntzkp.commit_time*10^6
        total_challenge_time += ntzkp.challenge_time*10^6
        total_prove_time += ntzkp.prove_time*10^6
        total_verify_time += ntzkp.verify_time*10^6
    else:
        pass

print("\n-----------------------------")
print("Total keygen time / Average keygen time       : %.2fus / %.2fus" % (total_keygen_time, total_keygen_time/iteration))
print("Total commit time / Average commit time       : %.2fus / %.2fus" % (total_commit_time, total_commit_time/iteration))
print("Total challenge time / Average challenge time : %.2fus / %.2fus" % (total_challenge_time, total_challenge_time/iteration))
print("Total prove time / Average prove time         : %.2fus / %.2fus" % (total_prove_time, total_prove_time/iteration))
print("Total verify time / Average verify time       : %.2fus / %.2fus" % (total_verify_time, total_verify_time/iteration))

iteration : 20 / 20
-----------------------------
Total keygen time / Average keygen time       : 969304225.68ms / 48465211.28ms
Total commit time / Average commit time       : 386441.95ms / 19322.10ms
Total challenge time / Average challenge time : 544.79ms / 27.24ms
Total prove time / Average prove time         : 77632.67ms / 3881.63ms
Total verify time / Average verify time       : 615432.02ms / 30771.60ms
